# Финальный проект: корпус дневников
### Авторы проекта: Вероника Ганеева, Алла Горбунова, Елизавета Клыкова (БКЛ181)
## Описание проекта
Данные для корпуса собраны с сайта diary.ru c помощью краулера и записаны в pandas dataframe. Морфологическая обработка выполнена с помощью Mystem.

Для корректной работы программы в одной папке с ней должны находиться mystem.exe и chromedriver.exe совместимой с вашим браузером версии (скачать можно вот тут: https://sites.google.com/a/chromium.org/chromedriver/downloads).

**ВАЖНО: не запускать через Restart & Run All, сначала первые 4 ячейки, пройти авторизацию и дальше запускать остаток программы!!!**

## Шаг 1: сбор данных

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import os
import re
import ast
import json
import pandas as pd
from tqdm.auto import tqdm
from nltk import sent_tokenize
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [3]:
driver = webdriver.Chrome('chromedriver.exe')

На сайте требуется авторизация при первом входе (я делаю это вручную, т.к. не хочу распространять свои логин и пароль). Запрашиваем первую страницу, на открывшейся странице "Ошибка доступа" вводим логин и пароль, нажимаем "Войти".

In [4]:
page = driver.get(f'https://www.diary.ru/?last_post&from=0')

Теперь с помощью функции получаем ссылки на страницы записей, исключая те, что заканчиваются на "closed.htm": они ведут на страницы с закрытым доступом, куда попасть могут только подписанные на дневник пользователи.

In [5]:
def get_links(page_num):
    page_number = 0
    links = []
    for i in tqdm(range(page_num)):
        page = driver.get(
            f'https://www.diary.ru/?last_post&from={page_number}')
        links_web = driver.find_elements_by_xpath("//div[@class='left']/a")
        for wlink in links_web:
            link = wlink.get_attribute('href')
            if link:
                if not link.endswith('_closed.htm'):
                    links.append(link)
        page_number += 20
    return links

In [6]:
links = get_links(30)
len(links)

421

Функция ниже получает информацию о постах и тексты по полученным выше ссылкам. Я использую try - except NoSuchElementException: это внутренняя ошибка Selenium'а. Избежать такой проверки никак нельзя, поскольку посты имеют разный формат.

Информация о каждом тексте записывается в список в таком порядке: ссылка, id поста, автор, название, полный текст. Если какого-то из элементов нет, на его место записывается пустая строка (это возможно для названия - не всегда пост имеет заголовок). Таким образом, все списки имеют равную длину и ошибки при обращении по индексу исключены.

Готовый список с информацией о тексте записывается в общий список all_texts, который и возвращает функция. Кроме того, добавлена простая проверка длины текста: текст разбивается по пробелоподобным символам, затем проверяется, содержит ли он хотя бы 100 слов, и только при положительном ответе данные поста записываются в общий список.

In [7]:
def get_texts(links):
    all_texts = []
    for link in tqdm(links):
        text_info = []
        text_info.append(link)
        page = driver.get(link)
        post_id = driver.find_element_by_class_name(
            'singlePost').get_attribute('id').strip('post')
        text_info.append(post_id)
        try:
            author = driver.find_element_by_class_name('authorName').text
        except NoSuchElementException:
            author = 'Unknown'
        text_info.append(author)
        try:
            title = driver.find_element_by_xpath(
                "//div[@class='postTitle header']/h1").text
        except NoSuchElementException:
            title = driver.find_element_by_xpath("//a[@class='title']").text
        text_info.append(title)
        try:
            text = driver.find_element_by_class_name('postInner').text
        except NoSuchElementException:
            text = driver.find_element_by_xpath(
                "//div[@class='post-inner']").text
        text_info.append(text)
        if text:
            if len(text.split()) >= 100:
                all_texts.append(text_info)
    return all_texts

In [8]:
texts = get_texts(links)

In [9]:
len(texts)

133

Полученные данные записываем в датафрейм, а заодно сохраняем в tsv-файл.

In [10]:
column_names = ['link', 'post_id', 'author', 'title', 'text']

In [11]:
df = pd.DataFrame(texts, columns=column_names)
df

,link,post_id,author,title,text
0,https://minnskriget.diary.ru/p220050852.htm,220050852,Alexi Kivilaakso,,"Ну снова здорово!!!! Снова просыпаешься, идешь..."
1,https://liss-rin.diary.ru/p220050837_nablyuden...,220050837,все записи пользователя в сообществе\nБратья М...,Наблюдения.,"Т.н. ""вспышка"" заболевания связана с нескольки..."
2,https://vsevse-ru.diary.ru/p220050835_grushevy...,epigraph,все записи пользователя в сообществе\nvsevse.ru,Грушевый салат с рукколой и овечьим сыром,Грушевый салат с рукколой и овечьим сыром\nЦик...
3,https://freezone-3.diary.ru/p220050833.htm,220050833,Дина_Мит,,Сижу и мысленно хвалю и глажу себя по головке ...
4,https://avelis.diary.ru/p220050831.htm,220050831,Avelis,,"в статье на висиру все так просто и понятно, н..."
...,...,...,...,...,...
128,https://mizeri.diary.ru/p220048990_zapishu-na-...,220048990,Mizerikord,запишу на память.,"Вот чем полезны бывают дайры. Полезла в ""прошл..."
129,https://ladojka.diary.ru/p220048976.htm,220048976,Nagi Taicho,,"Наткнулся на этот твит, сижу смеюсь. Это я - я..."
130,https://inaja-line.diary.ru/p220048960_kosmeto...,epigraph,gavrusssha,косметос,"я еще немного попизжу, отчасти для собственной..."
131,https://art-blog.diary.ru/p220048935.htm,220048935,Никипенок,,Продажи-пост!\nУ нас наконец-то готовы хэллоуи...


In [12]:
df.to_csv('texts.tsv', sep='\t', index=False)

Не забываем выйти из сессии:

In [13]:
driver.quit()

## Шаг 2: морфологическая обработка
Считываем файл с текстами и информацией о них в новый датафрейм, тексты записываем в список texts для обработки.

In [14]:
df2 = pd.read_csv('texts.tsv', sep='\t')
texts = list(df2['text'])

Mystem был выбран из-за его точности, его консольный вариант - из-за скорости работы. Способ ниже неоптимален с точки зрения времени, т.к. распарсить целый текст было бы быстрее, но тогда возникают сложности однозначного разделения текстов, а это принципиальный для создания корпуса момент. Таким образом, мы решили пожертвовать скоростью ради точности.

In [15]:
def parse_with_mystem(texts):
    parsed_texts = []
    for text in tqdm(texts):
        sents = sent_tokenize(text)
        parsed_sents = []
        for sent in sents:
            with open('raw_text.txt', 'w', encoding='utf-8') as f:
                f.write(sent)
            os.system(
                r'.\mystem.exe -i -d --format json raw_text.txt parsed_text.json')
            json_list = []
            with open('parsed_text.json', encoding='utf-8') as f:
                for line in f.readlines():
                    json_list.append(json.loads(line))
            parsed_sents.append(json_list)
        parsed_texts.append(parsed_sents)
    return parsed_texts

10:80: E501 line too long (82 > 79 characters)


In [16]:
parsed_texts = parse_with_mystem(texts)

Размеченные тексты нужно преобразовать в удобный для поиска формат, в т.ч. приведя теги в более читаемый вид:

In [17]:
rule = {
    'S': 'NOUN',
    'A': 'ADJ',
    'V': 'VERB',
    'ADVPRO': 'ADV',
    'PR': 'PREP',
    'PART': 'PRCL',
    'APRO': 'PRON',
    'SPRO': 'PRON',
}

In [18]:
def unify(tag):
    if tag in rule:
        return rule[tag]
    else:
        return tag

В нашем датафрейме отдельно будут храниться тексты, состоящие из слов с их разборами, и тексты, состоящие из предложений, в которых все слова также содержат информацию о разборе. Каждый текст сначала токенизируем с помощью NLTK, а затем обрабатываем по предложению. Это позволяет установить однозначное соответствие между предложением полного текста и разбором этого предложения, хотя с точки зрения скорости неоптимально. Мы решили воспользоваться этим способом, поскольку пользователя это никак не затрагивает: его запросы обрабатываются иначе и поиск работает быстро.

In [19]:
def get_word_info(texts):
    tagged_texts = []
    tagged_sent_texts = []
    for text in parsed_texts:
        parsed_text = []
        parsed_sents = []
        for sent in text:
            parsed_sent = []
            full_sent = []
            for part in sent:
                full_sent.extend(part)
            for word in full_sent:
                if word['analysis']:  # может быть пустое
                    form = word['text']
                    grammar = word['analysis'][0]['gr']
                    pos = unify(grammar.split('=')[0].split(',')[0])
                    lemma = word['analysis'][0]['lex']
                    parsed_word = (form, lemma, pos)
                    parsed_sent.append(parsed_word)
                    parsed_text.append(parsed_word)
            # на этом этапе разрешаем пустые предложения
            parsed_sents.append(parsed_sent)
        tagged_texts.append(parsed_text)
        tagged_sent_texts.append(parsed_sents)
    return tagged_texts, tagged_sent_texts

In [20]:
tagged_texts, tagged_sent_texts = get_word_info(parsed_texts)

Теперь нужно разбить тексты на предложения. Поскольку часть предложений Mystem возвращает пустыми (например, если они содержат только цифры), мы сравниваем кол-во предложений сырого и обработанного текстов и удаляем по индексу те предложения необработанного текста, для который Mystem не сделал разбора.

In [21]:
sent_texts = [sent_tokenize(t) for t in texts]

In [22]:
def get_clean_sents(sent_texts, tagged_sent_texts):
    clean_sent_texts = []
    clean_tagged_sents = []
    for n, text in enumerate(tagged_sent_texts):
        text_sts = []
        tagged_sts = []
        for i, sent in enumerate(text):
            if sent:
                tagged_sts.append(sent)
                text_sts.append(sent_texts[n][i])
        clean_sent_texts.append(text_sts)
        clean_tagged_sents.append(tagged_sts)
    return clean_sent_texts, clean_tagged_sents

In [23]:
clean_sent_texts, clean_tagged_sents = get_clean_sents(
    sent_texts, tagged_sent_texts)

Наконец, запишем данные в датафрейм:

In [24]:
df2['sentences'] = clean_sent_texts
df2['parsed_text'] = tagged_texts
df2['sent_text'] = clean_tagged_sents
df2

,link,post_id,author,title,text,sentences,parsed_text,sent_text
0,https://minnskriget.diary.ru/p220050852.htm,220050852,Alexi Kivilaakso,NaN,"Ну снова здорово!!!! Снова просыпаешься, идешь...","[Ну снова здорово!!!!, Снова просыпаешься, иде...","[(Ну, ну, PRCL), (снова, снова, ADV), (здорово...","[[(Ну, ну, PRCL), (снова, снова, ADV), (здоров..."
1,https://liss-rin.diary.ru/p220050837_nablyuden...,220050837,все записи пользователя в сообществе\nБратья М...,Наблюдения.,"Т.н. ""вспышка"" заболевания связана с нескольки...","[Т.н., ""вспышка"" заболевания связана с несколь...","[(Т, т, NOUN), (н, н, NOUN), (вспышка, вспышка...","[[(Т, т, NOUN), (н, н, NOUN)], [(вспышка, вспы..."
2,https://vsevse-ru.diary.ru/p220050835_grushevy...,epigraph,все записи пользователя в сообществе\nvsevse.ru,Грушевый салат с рукколой и овечьим сыром,Грушевый салат с рукколой и овечьим сыром\nЦик...,[Грушевый салат с рукколой и овечьим сыром\nЦи...,"[(Грушевый, грушевый, ADJ), (салат, салат, NOU...","[[(Грушевый, грушевый, ADJ), (салат, салат, NO..."
3,https://freezone-3.diary.ru/p220050833.htm,220050833,Дина_Мит,NaN,Сижу и мысленно хвалю и глажу себя по головке ...,[Сижу и мысленно хвалю и глажу себя по головке...,"[(Сижу, сидеть, VERB), (и, и, CONJ), (мысленно...","[[(Сижу, сидеть, VERB), (и, и, CONJ), (мысленн..."
4,https://avelis.diary.ru/p220050831.htm,220050831,Avelis,NaN,"в статье на висиру все так просто и понятно, н...","[в статье на висиру все так просто и понятно, ...","[(в, в, PREP), (статье, статья, NOUN), (на, на...","[[(в, в, PREP), (статье, статья, NOUN), (на, н..."
...,...,...,...,...,...,...,...,...
128,https://mizeri.diary.ru/p220048990_zapishu-na-...,220048990,Mizerikord,запишу на память.,"Вот чем полезны бывают дайры. Полезла в ""прошл...","[Вот чем полезны бывают дайры., Полезла в ""про...","[(Вот, вот, PRCL), (чем, что, PRON), (полезны,...","[[(Вот, вот, PRCL), (чем, что, PRON), (полезны..."
129,https://ladojka.diary.ru/p220048976.htm,220048976,Nagi Taicho,NaN,"Наткнулся на этот твит, сижу смеюсь. Это я - я...","[Наткнулся на этот твит, сижу смеюсь., Это я -...","[(Наткнулся, наткнуться, VERB), (на, на, PREP)...","[[(Наткнулся, наткнуться, VERB), (на, на, PREP..."
130,https://inaja-line.diary.ru/p220048960_kosmeto...,epigraph,gavrusssha,косметос,"я еще немного попизжу, отчасти для собственной...","[я еще немного попизжу, отчасти для собственно...","[(я, я, PRON), (еще, еще, ADV), (немного, немн...","[[(я, я, PRON), (еще, еще, ADV), (немного, нем..."
131,https://art-blog.diary.ru/p220048935.htm,220048935,Никипенок,NaN,Продажи-пост!\nУ нас наконец-то готовы хэллоуи...,"[Продажи-пост!, У нас наконец-то готовы хэллоу...","[(Продажи, продажа, NOUN), (пост, пост, NOUN),...","[[(Продажи, продажа, NOUN), (пост, пост, NOUN)..."


Сохраняем наш недокорпус в новый файл .tsv.

In [25]:
df2.to_csv('corpus.tsv', sep='\t', index=False)